In [131]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score, f1_score
)
import numpy as np
import pandas as pd
import sys, os

# ====== YOUR PREPROCESSOR ======
sys.path.append(os.path.abspath("../.."))
from src.preprocessing import preprocess_text_fin


In [132]:
DATA_PATH = "../../data/data.csv"
RANDOM_STATE = 42

# 1) Cấu hình chung (áp dụng cho mọi mô hình)

**Lý do thiết kế TF-IDF & CV:**

* **`ngram_range=(1,2)`**: câu ngắn → bigram bắt cụm giàu tín hiệu (“rose sharply”, “beat estimates”).
* **`min_df∈{3,5,10}`**: lọc từ quá hiếm do tập chỉ \~5.8k câu.
* **`max_df∈{0.9,0.95}`**: loại từ quá phổ biến (gần stopword).
* **`max_features∈{5k,10k,15k}`**: cân bằng bias/variance với cỡ dữ liệu \~5.8k.
* **`sublinear_tf=True`**: giảm “burstiness” của từ xuất hiện nhiều.
* **`norm='l2'`**: phù hợp linear models trên dữ liệu sparse.

In [133]:
# Không chỉnh tokenizer trong grid để tránh lỗi set_params
BASE_TFIDF = TfidfVectorizer(
    preprocessor=preprocess_text_fin,
    tokenizer=word_tokenize,   # gắn với tiền xử lý text tiếng Anh
    token_pattern=None,
    use_idf=True
)

tfidf_space = {
    # (1,1) giữ unigram cho từ khóa đơn; (1,2) thêm bigram để bắt cụm giàu ngữ nghĩa
    # vì câu ngắn nên không dùng trigram để tránh loãng & overfit.
    "tfidf__ngram_range": [(1,1), (1,2)],

    # Lọc từ quá hiếm (typo/nhiễu) vì data chỉ ~5.8k câu → giảm chiều & variance.
    "tfidf__min_df": [3, 5, 10],

    # Loại từ quá phổ biến (gần stopword) nhưng KHÔNG xóa stopwords cứng
    # để giữ phủ định / đơn vị tài chính (e.g., "not", "%", "per cent").
    "tfidf__max_df": [0.90, 0.95],

    # Cân bằng giữa độ phủ từ vựng tài chính và nguy cơ overfit/chi phí tính.
    "tfidf__max_features": [5000, 10000, 15000],

    # Giảm “burstiness” khi một từ lặp nhiều trong cùng văn bản → trọng số ổn định hơn.
    "tfidf__sublinear_tf": [True],

    # L2 giúp linear models hoạt động tốt trên vector sparse, chuẩn hóa độ dài tài liệu.
    "tfidf__norm": ["l2"],
}

# 2) Logistic Regression + TF-IDF (baseline mạnh)

**Lý do chọn & tham số:**

* LR rất hợp với vector TF-IDF high-dimensional; nhanh, ổn định.
* **`C`** điều khiển regularization: dữ liệu 5.8k → cần quét từ chặt đến lỏng.
* **`solver='liblinear'` + `multi_class='ovr'`**: ổn với sparse và 3 lớp.
* **`class_weight='balanced'`**: bù lệch lớp

In [134]:
from sklearn.linear_model import LogisticRegression

pipe_lr = Pipeline([
    ("tfidf", BASE_TFIDF),
    ("clf", OneVsRestClassifier(
        LogisticRegression(
            penalty="l2",                                    # L2 ổn định trên high-dimensional TF-IDF
            solver="liblinear",                              # Phù hợp sparse + multi-class OVR quy mô vừa
            class_weight="balanced",                         # Cân lệch lớp 54/32/14 theo 1/freq
            max_iter=2000,                                   # Đảm bảo hội tụ khi feature nhiều (bigram, 10k+)
            random_state=RANDOM_STATE
        ),
        n_jobs=-1
    ))
])


param_grid_lr = {
    **tfidf_space,
    # Quét từ regularization mạnh → yếu để tìm điểm cân bằng bias/variance với ~5.8k câu
    "clf__estimator__C": [0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
}

# 3) Linear SVM (LinearSVC) + TF-IDF (mạnh với dữ liệu sparse)

**Lý do & tham số:**

* Linear SVM thường cho biên tốt trên TF-IDF.
* **`C`** quét rộng để tìm margin tối ưu.
* **`loss='squared_hinge'`** là chuẩn cho LinearSVC.
* **`class_weight='balanced'`** xử lý lệch.


In [135]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
pipe_svm = Pipeline([
    ("tfidf", BASE_TFIDF),
    ("clf", CalibratedClassifierCV(
        estimator=LinearSVC(
            class_weight="balanced",   # bù lệch lớp 54/32/14 theo 1/freq
            max_iter=5000,             # nhiều feature → cần đủ vòng để hội tụ
            random_state=42          # thêm nếu muốn tái lập
        ),
        cv=3,                          # ⚠️ calibration cũng CV → giảm từ 5 xuống 3 để tiết kiệm thời gian
        method="sigmoid"               # Platt scaling: nhanh, ổn định; đủ tốt cho 5.8k mẫu
    ))
])
param_grid_svm = {
    **tfidf_space,
    # Điều chỉnh độ rộng margin; quét rộng để tránh under/over-regularize
    "clf__estimator__C": [0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
}

# 4) Multinomial Naive Bayes + TF-IDF (baseline cực nhanh)

**Lý do & tham số:**

* MNB phù hợp xác suất từ; nhanh, nhẹ để làm baseline.
* **`alpha`** (Laplace/Lidstone) rất quan trọng vì từ hiếm xuất hiện nhiều trong text ngắn → quét log-scale.
* **`fit_prior`** bật/tắt để xem tác động của phân bố lớp lệch.


In [136]:
from sklearn.naive_bayes import MultinomialNB

pipe_mnb = Pipeline([
    ("tfidf", BASE_TFIDF),
    ("clf", MultinomialNB()) # Baseline rất nhanh cho text; dùng xác suất từ

])

param_grid_mnb = {
    **tfidf_space,
    # Làm mượt xác suất cho từ hiếm (rất phổ biến trong câu ngắn) → quét log-scale
    "clf__alpha": [0.01, 0.05, 0.1, 0.5, 1.0],
    # Thử dùng/không dùng prior vì dữ liệu lệch lớp; xem prior có giúp ổn định hay làm thiên lệch
    "clf__fit_prior": [True, False],
}

# 5) Random Forest + TF-IDF

**Lý do & tham số:**

* RF không lý tưởng nếu **chỉ** TF-IDF (quá high-dim). Tuy nhiên, nếu sau này anh **gộp thêm feature thống kê** (độ dài câu, số tiền, có/không dấu %) thì RF phát huy.
* Dù vậy, để hoàn chỉnh benchmark, ta vẫn cho grid gọn, có **`class_weight='balanced_subsample'`** để bù lệch.


In [137]:
from sklearn.ensemble import RandomForestClassifier

pipe_rf = Pipeline([
    ("tfidf", BASE_TFIDF),
    ("clf", RandomForestClassifier(
        n_jobs=-1,
        class_weight="balanced_subsample",
        random_state=42
    ))
])

param_grid_rf = {
    **tfidf_space,
}

## 6) Gradient Boosting + TF-IDF (boosting cây nhẹ)

**Lý do & tham số:**

* GBC thường kém thuận lợi hơn linear models trên TF-IDF thuần, nhưng đáng thử cho benchmark; nếu thêm feature thống kê/khác miền, GBC cải thiện rõ.

* Không có class_weight, ta regularize bằng subsample + min_samples_leaf để bớt overfit lớp lớn.

In [138]:
from sklearn.ensemble import GradientBoostingClassifier

pipe_gb = Pipeline([
    ("tfidf", BASE_TFIDF),
    ("clf", GradientBoostingClassifier(
        random_state=RANDOM_STATE
    ))
])

param_grid_gb = {
    **tfidf_space,
    # Cặp (learning_rate, n_estimators) điều khiển bias/variance:
    # lr nhỏ + nhiều cây → tổng quát hơn; lr lớn + ít cây → nhanh nhưng dễ overfit.
    # "clf__n_estimators": [200, 400],
    # "clf__learning_rate": [0.05, 0.1],
    # # Cây nông (2–3) phù hợp dữ liệu nhiễu/sparse, tránh học quan hệ giả
    # "clf__max_depth": [2, 3],
    # # Stochastic boosting (subsample<1) giảm variance & cải thiện tổng quát hóa
    # "clf__subsample": [0.8, 1.0],
    # # Tránh lá quá nhỏ dễ bám nhiễu của lớp lớn
    # "clf__min_samples_leaf": [1, 2],
    # # Giới hạn số feature mỗi split để bớt nhiễu khi high-dim TF-IDF
    # "clf__max_features": [None, "sqrt"],
}

# 5) GridSearchCV

In [139]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
def run_gs(pipe, grid, X_train, y_train):
    gs = GridSearchCV(
        estimator=pipe,
        param_grid=grid,
        scoring="f1_macro",
        cv=cv,
        n_jobs=-1,
        verbose=1
    )
    gs.fit(X_train, y_train)
    return gs

# 6) Preprocessing 

In [140]:
def preprocess_csv(csv_file: str) -> pd.DataFrame:
    df = pd.read_csv(csv_file)
    df["labels"] = LabelEncoder().fit_transform(df["Sentiment"])
    df.drop_duplicates(subset=["Sentence"], keep="first", inplace=True)
    # df["Sentence"] = df["Sentence"].astype(str).apply(preprocess_text_fin)
    return df

In [141]:
df = preprocess_csv(DATA_PATH)
X_train, X_test, y_train, y_test = train_test_split(
    np.array(df["Sentence"]),
    np.array(df["labels"]), 
    test_size=0.25, 
    random_state=42
    )

In [142]:

gs_lr = run_gs(pipe_lr, param_grid_lr, X_train, y_train)

Fitting 5 folds for each of 252 candidates, totalling 1260 fits


In [143]:
gs_svm = run_gs(pipe_svm, param_grid_svm, X_train, y_train)

Fitting 5 folds for each of 252 candidates, totalling 1260 fits


In [144]:
gs_mnb = run_gs(pipe_mnb, param_grid_mnb, X_train, y_train)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


In [145]:
gs_rf = run_gs(pipe_rf, param_grid_rf, X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


In [146]:
gs_gb = run_gs(pipe_gb, param_grid_gb, X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


In [147]:
import numpy as np
import pandas as pd
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    classification_report, confusion_matrix, roc_auc_score
)
from sklearn.utils.multiclass import unique_labels

# 1) Tóm tắt kết quả Cross-Validation (best) của nhiều GridSearch
def summarize_cv_results(gridsearch_dict):
    rows = []
    for name, gs in gridsearch_dict.items():
        rows.append({
            "model": name,
            "cv_best_f1_macro": gs.best_score_,
            "best_params": gs.best_params_
        })
    df = pd.DataFrame(rows).sort_values("cv_best_f1_macro", ascending=False)
    print("\n=== CV Best Summary (sorted by f1_macro) ===")
    print(df.to_string(index=False))
    return df

# 2) Đánh giá chi tiết trên tập test cho 1 GridSearch đã fit
def evaluate_on_test(gs, X_test, y_test, model_name="Model"):
    best_est = gs.best_estimator_
    y_pred   = best_est.predict(X_test)

    # Thử lấy score cho ROC-AUC macro (ưu tiên predict_proba, fallback decision_function)
    y_score = None
    has_proba = hasattr(best_est, "predict_proba")
    has_decision = hasattr(best_est, "decision_function")
    if has_proba:
        try:
            y_score = best_est.predict_proba(X_test)
        except Exception:
            y_score = None
    if (y_score is None) and has_decision:
        try:
            y_score = best_est.decision_function(X_test)
        except Exception:
            y_score = None

    # Metrics cơ bản
    acc   = accuracy_score(y_test, y_pred)
    f1m   = f1_score(y_test, y_pred, average="macro")
    precm = precision_score(y_test, y_pred, average="macro", zero_division=0)
    recm  = recall_score(y_test, y_pred, average="macro", zero_division=0)

    # ROC-AUC macro (nếu có y_score nhiều lớp)
    rocauc = None
    if y_score is not None:
        try:
            rocauc = roc_auc_score(y_test, y_score, multi_class="ovo", average="macro")
        except Exception:
            rocauc = None

    # In tổng quan
    print(f"\n================= {model_name} =================")
    print("Best params:", gs.best_params_)
    print(f"CV best f1_macro: {gs.best_score_:.4f}")
    print(f"Test Accuracy   : {acc:.4f}")
    print(f"Test F1-macro   : {f1m:.4f}")
    print(f"Test Precision-m: {precm:.4f}")
    print(f"Test Recall-m   : {recm:.4f}")
    if rocauc is not None:
        print(f"Test ROC-AUC-m  : {rocauc:.4f}")

    # Classification report
    print("\nClassification report:")
    print(classification_report(y_test, y_pred, digits=4, zero_division=0))

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    labels = unique_labels(y_test, y_pred)
    print("Confusion matrix (rows=true, cols=pred):")
    print(pd.DataFrame(cm, index=[f"true_{l}" for l in labels],
                          columns=[f"pred_{l}" for l in labels]).to_string())
    return {
        "model": model_name,
        "cv_best_f1_macro": gs.best_score_,
        "test_accuracy": acc,
        "test_f1_macro": f1m,
        "test_precision_macro": precm,
        "test_recall_macro": recm,
        "test_roc_auc_macro": rocauc,
        "best_params": gs.best_params_
    }

# 3) Chạy đánh giá cho tất cả mô hình + bảng tổng hợp test
def evaluate_all(gs_dict, X_test, y_test):
    _ = summarize_cv_results(gs_dict)
    test_rows = []
    for name, gs in gs_dict.items():
        res = evaluate_on_test(gs, X_test, y_test, model_name=name)
        test_rows.append(res)
    test_df = pd.DataFrame(test_rows).sort_values("test_f1_macro", ascending=False)
    print("\n=== Test Summary (sorted by f1_macro) ===")
    print(test_df[["model","cv_best_f1_macro","test_f1_macro","test_accuracy","test_precision_macro","test_recall_macro","test_roc_auc_macro"]].to_string(index=False))
    return test_df


## ✅ Kết luận tổng quan

* **Mô hình tốt nhất**: **Logistic Regression (LR)**

  * **CV F1-macro**: 0.6517 → **Test F1-macro**: **0.6943** (cao nhất)
  * **Test Accuracy**: 0.7423 | **ROC-AUC macro**: 0.8774 (khả năng xếp hạng tốt → tối ưu ngưỡng còn dư địa)
* **Thứ hạng tiếp theo**: **MultinomialNB** (0.6581) ≈ **LinearSVM** (0.6488) > **RandomForest** (0.6209) ≈ **GradBoost** (0.6176)

## 📌 Diễn giải nhanh theo lớp

* **LR** cân bằng tốt nhất giữa các lớp; **lớp 0** vẫn là điểm yếu của mọi mô hình nhưng LR có **recall lớp 0 \~0.60** (tốt hơn SVM/RF/GB).
* **SVM** accuracy tương đương LR nhưng **macro-F1 thấp** do **recall lớp 0 sụt (0.315)**.
* **MNB** là baseline nhanh và khá cân bằng, nhưng kém LR một chút.
* **RF/GBoost** cho accuracy ổn nhưng **macro-F1 thấp** — đặc trưng của cây trên TF-IDF sparse.


In [152]:
gs_dict = {
    "Logistic Regression": gs_lr,
    "LinearSVM": gs_svm,
    "MultinomialNB": gs_mnb,
    "RandomForest": gs_rf,
    "GradBoost": gs_gb,
}
test_summary = evaluate_all(gs_dict, X_test, y_test)


=== CV Best Summary (sorted by f1_macro) ===
              model  cv_best_f1_macro                                                                                                                                                                                         best_params
Logistic Regression          0.665786                   {'clf__estimator__C': 2.0, 'tfidf__max_df': 0.9, 'tfidf__max_features': 5000, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2), 'tfidf__norm': 'l2', 'tfidf__sublinear_tf': True}
      MultinomialNB          0.636856 {'clf__alpha': 0.5, 'clf__fit_prior': False, 'tfidf__max_df': 0.9, 'tfidf__max_features': 5000, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 1), 'tfidf__norm': 'l2', 'tfidf__sublinear_tf': True}
          LinearSVM          0.617239                   {'clf__estimator__C': 0.5, 'tfidf__max_df': 0.9, 'tfidf__max_features': 5000, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2), 'tfidf__norm': 'l2', 'tfidf__sublinear_tf': True}
       RandomFores

In [153]:
from pathlib import Path
from joblib import dump
import time

# === Thư mục lưu ===
SAVE_DIR = Path("../../models")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

def save_selfcontained(name: str, model_or_gs, with_ts: bool = False):
    pipe = model_or_gs.best_estimator_ if hasattr(model_or_gs, "best_estimator_") else model_or_gs
    assert hasattr(pipe, "predict"), "Object phải là pipeline/estimator đã fit."
    suffix = f"_{time.strftime('%Y%m%d-%H%M%S')}" if with_ts else ""
    path = SAVE_DIR / f"selfcontained_{name}{suffix}.joblib"
    dump(pipe, path, compress=3)
    print(f"✅ Saved: {path}")

# === Lưu tất cả mô hình ===
save_selfcontained("logreg",        gs_lr)        # Logistic Regression
save_selfcontained("linearsvm_cal", gs_svm)       # LinearSVC đã bọc CalibratedClassifierCV
save_selfcontained("mnb",           gs_mnb)       # MultinomialNB
save_selfcontained("rf",            gs_rf)        # RandomForest
save_selfcontained("gboost",        gs_gb)        # GradientBoosting

✅ Saved: ..\..\models\selfcontained_logreg.joblib
✅ Saved: ..\..\models\selfcontained_linearsvm_cal.joblib
✅ Saved: ..\..\models\selfcontained_mnb.joblib
✅ Saved: ..\..\models\selfcontained_rf.joblib
✅ Saved: ..\..\models\selfcontained_gboost.joblib
